In [1]:
import pandas as pd  # 데이터 처리 및 CSV 파일 I/O (입출력)을 위한 라이브러리
import numpy as np  # 수치 계산을 위한 라이브러리
from transformers import (
    AutoTokenizer,  # 사전 훈련된 토크나이저를 자동으로 불러오기 위한 함수
    AutoConfig,  # 사전 훈련된 모델 설정을 자동으로 불러오기 위한 함수
    AutoModelForCausalLM,  # 인과 관계 언어 모델 (Causal Language Model)을 위한 사전 훈련된 모델을 자동으로 불러오기 위한 함수
    DataCollatorWithPadding,  # 배치 내의 데이터를 자동으로 패딩하기 위한 함수
    TrainingArguments,  # 트레이닝 관련 설정을 위한 클래스
    Trainer,  # 모델을 학습시키기 위한 간단한 API
    BitsAndBytesConfig,  # Bit and Bytes 관련 설정을 위한 클래스
    AutoModelForSeq2SeqLM,  # 시퀀스-투-시퀀스(sequence-to-sequence) 언어 모델을 위한 사전 훈련된 모델을 자동으로 불러오기 위한 함수
    Seq2SeqTrainingArguments,  # 시퀀스-투-시퀀스 학습 관련 설정을 위한 클래스
    Seq2SeqTrainer  # 시퀀스-투-시퀀스 모델을 학습시키기 위한 클래스
)
import transformers  # Transformer 모델 관련 라이브러리
from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig, prepare_model_for_kbit_training  # 효율적인 트레이닝을 위한 함수 및 클래스
import torch  # PyTorch 라이브러리
import time  # 시간 측정을 위한 라이브러리

bnb_config = BitsAndBytesConfig(
   load_in_8bit=True,  # 8비트로 모델을 로드함 (메모리 절약)
)

new_model = "01-ai/Yi-34B-Chat"  # 사용할 사전 훈련된 모델 이름

tokenizer = AutoTokenizer.from_pretrained(new_model)  # 사전 훈련된 모델의 토크나이저 불러오기
tokenizer.pad_token = tokenizer.eos_token  # 패드 토큰을 EOS 토큰으로 설정
tokenizer.padding_side = "right"  # 패딩을 오른쪽에 적용

model = AutoModelForCausalLM.from_pretrained(new_model,
                                             device_map ='auto',  # 모델을 로드할 디바이스를 자동으로 설정 （4개의 GPU에 걸쳐 균형 잡힌 모델 로드）
                                             quantization_config=bnb_config,  # 양자화 설정 적용
                                             use_cache=True,  # 캐시 사용
                                            )



def generate_response(prompt, model):
    encoded_input = tokenizer(prompt,  return_tensors="pt", add_special_tokens=True)  # 입력을 토크나이징해서 텐서로 변환
    model_inputs = encoded_input.to('cuda')  # 모델 입력을 CUDA 장치로 보냄
    
    generated_ids = model.generate(**model_inputs,
                                 max_new_tokens=2560,  # 최대 생성 토큰 수
                                 do_sample=True,  # 샘플링을 통해 텍스트 생성
                                 pad_token_id=tokenizer.eos_token_id,  # PAD 토큰 ID 설정
                                )
    
    decoded_output = tokenizer.batch_decode(generated_ids)  # 생성된 토큰 ID를 다시 텍스트로 디코드
    
    return decoded_output[0].replace(prompt, "")  # 입력 문구를 제외한 생성된 응답 반환

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

In [2]:
# 사용자로부터 받은 메시지를 저장하는 리스트입니다.
messages = [
    {"role": "user", "content": "Hello, do you know Dong-A University?"}
]

# 토크나이저를 사용하여 대화를 토큰화하고 생성할 내용을 위한 프롬프트를 추가합니다.
# 'pt'는 PyTorch 텐서를 반환하도록 합니다.
input_ids = tokenizer.apply_chat_template(conversation=messages, 
                                          tokenize=True, 
                                          add_generation_prompt=True, 
                                          return_tensors='pt')

# 토큰화된 입력을 모델에 제공하여 응답을 생성합니다.
# 입력은 'cuda'를 사용하여 GPU로 이동합니다.
output_ids = model.generate(input_ids.to('cuda'))

# 생성된 응답에서 처음 입력된 대화 부분을 제외하고 디코드합니다.
# 특수 토큰은 건너뜁니다.
res = tokenizer.decode(output_ids[0][input_ids.shape[1]:], skip_special_tokens=True)

# 생성된 응답을 출력합니다.
print(res)

Yes, Dong-A University is a private university located in Busan, South Korea. It was established in 1947 and is one of the韩国较著名的综合性大学之一。 Dong-A University offers a wide range of undergraduate and graduate programs across various fields, including business, engineering, humanities, social sciences, and natural sciences. The university is known for its strong emphasis on research and has produced many notable alumni in various industries. If you have any specific questions about Dong-A University, I'll be happy to assist you further.
